<h3>Loading the initial data</h3>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb

In [2]:
ds = pd.read_excel('ml_bc_pipeline\\ml_project1_data.xlsx')
ds.set_index('ID', inplace=True)

<h3>Brief exploration</h3>

In [4]:
print(ds.info(), "\nResponse rate: {}%".format(round(100*ds["Response"].mean(), 2)))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 6862 to 8720
Data columns (total 28 columns):
Year_Birth             2240 non-null int64
Education              2240 non-null object
Marital_Status         2240 non-null object
Income                 2216 non-null float64
Kidhome                2240 non-null int64
Teenhome               2240 non-null int64
Dt_Customer            2240 non-null object
Recency                2240 non-null int64
MntWines               2240 non-null int64
MntFruits              2240 non-null int64
MntMeatProducts        2240 non-null int64
MntFishProducts        2240 non-null int64
MntSweetProducts       2240 non-null int64
MntGoldProds           2240 non-null int64
NumDealsPurchases      2240 non-null int64
NumWebPurchases        2240 non-null int64
NumCatalogPurchases    2240 non-null int64
NumStorePurchases      2240 non-null int64
NumWebVisitsMonth      2240 non-null int64
AcceptedCmp3           2240 non-null int64
AcceptedCmp4           22

How many rows can we delete?

In [5]:
len(ds)*0.03

67.2

Getting the indexes (idx) for categorical/dummy and numerical variables:

In [6]:
ds.columns

Index(['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

In [7]:
cat_or_dummy_vars=['Education','Marital_Status','Kidhome','Teenhome','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5',
                'AcceptedCmp1', 'AcceptedCmp2','Complain' ,'Response'] 

numerical_var=['Year_Birth','Income','Dt_Customer','Recency','MntWines','MntFruits','MntMeatProducts','MntFishProducts',
              'MntSweetProducts','MntGoldProds','NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases',
              'NumWebVisitsMonth']

### Fixing datatype consistencies:

In [4]:
#Turning dates into date type:
ds.Dt_Customer=pd.to_datetime(ds['Dt_Customer'].str.replace('-',''),format='%Y%m%d',errors='ignore')

### Detecting outliers:

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn import metrics
from scipy.stats import multivariate_normal
from sklearn.metrics.pairwise import euclidean_distances
import eif as iso

## AS ISO FOREST SÓ PODEM TER VALORES NUMÉRICOS E NÃO TOLERAM NANS

df2 = ds[numerical_var]
df2 = df2.drop(columns=["Dt_Customer"])
df2 = df2.dropna()

""" CONTAMINATION: The amount of contamination of the data set, i.e. the proportion of outliers in the data set. 
Used when fitting to define the threshold on the decision function. """
clf = IsolationForest(max_samples=100, contamination = 0.05, random_state=np.random.RandomState(42))
clf.fit(df2)
outliers_isoflorest = clf.predict(df2)

## CUIDADO AGAIN COM OS PARAMETROS
if_eif = iso.iForest(df2.values, ntrees = 100, sample_size = 256, ExtensionLevel = 2)
anomaly_scores = if_eif.compute_paths(X_in = df2.values)
anomaly_scores_sorted = np.argsort(anomaly_scores)

C:\Users\Filipa\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
C:\Users\Filipa\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [11]:
ds[numerical_var]

,Year_Birth,Income,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth
ID,,,,,,,,,,,,,,,
6862,1971,1730.0,2014-05-18,65,1,1,3,1,1,1,15,0,0,0,20
5376,1979,2447.0,2013-01-06,42,1,1,1725,1,1,1,15,0,28,0,1
11110,1973,3502.0,2013-04-13,56,2,1,1,0,0,1,0,0,0,0,14
9931,1963,4023.0,2014-06-23,29,5,0,1,1,1,1,15,0,0,0,19
10311,1969,4428.0,2013-10-05,0,16,4,12,2,4,321,0,25,0,0,1
3955,1965,4861.0,2014-06-22,20,2,1,1,1,0,1,0,0,0,0,14
9303,1976,5305.0,2013-07-30,12,12,4,7,1,3,5,0,1,0,0,13
5899,1950,5648.0,2014-03-13,68,28,0,6,1,1,13,1,1,0,0,20
4246,1982,6560.0,2013-12-12,2,67,11,26,4,3,262,0,1,0,1,17


<h3>Finding duplicated rows</h3>

With the target variable

In [12]:
np.sum(ds.duplicated())

182

In [13]:
ds[ds.duplicated(keep=False)].sort_values(ds.columns.tolist())

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
ID,,,,,,,,,,,,,,,,,,,,,
2968,1943,PhD,Divorced,48948.0,0,0,2013-02-01,53,437,8,...,6,1,0,0,0,0,0,3,11,1
8800,1943,PhD,Divorced,48948.0,0,0,2013-02-01,53,437,8,...,6,1,0,0,0,0,0,3,11,1
4939,1946,Graduation,Together,37760.0,0,0,2012-08-31,20,84,5,...,7,0,0,0,0,0,0,3,11,0
8652,1946,Graduation,Together,37760.0,0,0,2012-08-31,20,84,5,...,7,0,0,0,0,0,0,3,11,0
10562,1946,Master,Married,66835.0,0,0,2013-09-28,21,620,26,...,2,0,0,0,0,0,0,3,11,0
7881,1946,Master,Married,66835.0,0,0,2013-09-28,21,620,26,...,2,0,0,0,0,0,0,3,11,0
455,1946,PhD,Married,51012.0,0,0,2013-04-18,86,102,9,...,6,0,0,0,0,0,0,3,11,0
7592,1946,PhD,Married,51012.0,0,0,2013-04-18,86,102,9,...,6,0,0,0,0,0,0,3,11,0
2114,1946,PhD,Single,82800.0,0,0,2012-11-24,23,1006,22,...,3,0,0,1,1,0,0,3,11,1


In [14]:
ds[ds.duplicated(keep=False)].groupby(ds.columns.tolist()).size().reset_index().rename(columns={0:'records'}).sort_values(['Dt_Customer'])

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,records
164,1988,Graduation,Single,68655.0,0,0,2012-08-10,95,456,19,...,0,0,0,0,0,0,3,11,0,2
114,1974,Graduation,Married,67445.0,0,1,2012-08-12,63,757,80,...,0,0,0,0,0,0,3,11,0,3
130,1977,Graduation,Married,42014.0,1,0,2012-08-17,56,244,15,...,0,0,0,0,0,0,3,11,1,2
120,1975,Master,Single,46098.0,1,1,2012-08-18,86,57,0,...,0,0,0,0,0,0,3,11,0,2
106,1973,2n Cycle,Married,35688.0,2,1,2012-08-22,94,73,3,...,0,0,0,0,0,0,3,11,0,2
46,1961,Graduation,Married,60544.0,1,1,2012-08-25,92,201,2,...,0,0,0,0,0,0,3,11,0,2
156,1985,Graduation,Single,29760.0,1,0,2012-08-29,87,64,4,...,0,0,0,0,0,0,3,11,0,2
1,1946,Graduation,Together,37760.0,0,0,2012-08-31,20,84,5,...,0,0,0,0,0,0,3,11,0,2
165,1988,PhD,Married,67546.0,0,0,2012-08-31,90,864,134,...,0,0,0,0,0,0,3,11,0,2
41,1960,Graduation,Together,62807.0,0,1,2012-08-31,83,526,28,...,0,0,0,0,0,0,3,11,0,2


Without the target variable

In [15]:
np.sum(ds.drop(['Response'], axis=1).duplicated())

201

### ACTUALLY REMOVING DUPLICATES

In [4]:
"""Nós temos dois tipos de dados repetidos. Dados repetidos com o Response diferente e dados repetidos com o Response
igual. O que significa que temos de ter cuidado, porque têm de ser tratados de maneira diferente
Primeiramente vamos eliminar os dados que estão repetidos com Response diferente. No máximo temos 3 casos repetidos,
pelo que tendo 2 para 1, não representa uma diferença significativamente grande para ficar com 1 individuo do grupo que 
tem mais. Depois, com os dados que ficamos, vamos ver os que têm duplicados, e neste caso vamos ficar sempre com 
um deles. Por default, o primeiro que aparece."""

ds = pd.read_excel("ml_bc_pipeline\\ml_project1_data.xlsx")
ds = ds.drop(columns=["ID","Response"])
da = pd.read_excel("ml_bc_pipeline\\ml_project1_data.xlsx")
#Colunas do dataset numa lista e retirar a primeira, aka ID
colunas = list(da)
colunas.pop(0)
#Obter o count e a lista com os ID's dos repetidos com o target.
target_count = da.groupby(colunas)['ID'].count()
target_list = da.groupby(colunas)['ID'].apply(list)
target = pd.concat([target_count, target_list], axis=1)
target.columns = ['count', 'lista']
#Obter o count e a lista com os ID's dos repetidos sem o target.
no_target_count = da.groupby(list(ds))['ID'].count()
no_target_list = da.groupby(list(ds))['ID'].apply(list)
no_target = pd.concat([no_target_count, no_target_list], axis=1)
no_target.columns = ['count', 'lista']
## Comparar os resultados do "com target" e do "sem target" e fazer a intersecção dos mesmos. Porque se estão iguais 
## nos dois lados, significa que nunca há casos em que os Response são diferentes.
no_target_set = set(map(tuple, no_target.lista))
target_set = set(map(tuple, target.lista))
id_intercept=no_target_set.intersection(target_set)
id_intercept = list(id_intercept)
#Juntar os ID's numa lista para depois ficar apenas com estes casos
ids = []
for i in id_intercept:
    for j in i:
        ids.append(j)
#Fiz isto para evitar perder os IDS, usando um merge atraves do index
frame = pd.read_excel("ml_bc_pipeline\\ml_project1_data.xlsx")
frame = frame.loc[frame['ID'].isin(ids)]
sem_ID = frame.drop(columns=["ID"])
sem_ID = sem_ID.drop_duplicates(keep="first")
frame_ID = frame["ID"]
merged = sem_ID.merge(frame_ID.to_frame(), left_index=True, right_index=True, how='inner')

<h3>Exploring missing values</h3>

In [23]:
def missing_values_reporter(df):    
    na_count = df.isna().sum() 
    ser = na_count[na_count > 0]
    return pd.DataFrame({"N missings": ser, "p missings": ser.divide(df.shape[0])})

rm_df_miss = missing_values_reporter(ds)
rm_df_miss

,N missings,p missings
Income,24,0.010714


Only the <strong>Income</strong> feature has missing values.

### Feature Extraction - Business Insights

In [5]:
ds_new_var = ds.copy()

#TER CUIDADO, CONFIRMAR SE O NUM WEB PURCHASES TB É MENSAL
ds_new_var["Web_Purchases_Per_Visit"] = ds["NumWebPurchases"]/ds["NumWebVisitsMonth"]

ds_new_var["Total_Purchases"] = ds["NumWebPurchases"] + ds["NumCatalogPurchases"] + ds["NumStorePurchases"]

ds_new_var["RatioWebPurchases"] = ds["NumWebPurchases"] / ds_new_var["Total_Purchases"]
ds_new_var["RatioCatalogPurchases"] = ds["NumCatalogPurchases"] / ds_new_var["Total_Purchases"]
ds_new_var["RatioStorePurchases"] = ds["NumStorePurchases"] / ds_new_var["Total_Purchases"]

#Getting today's date
import datetime
now = datetime.datetime.now()
ds_new_var["NumberYearsCostumer"] = now.year - ds['Dt_Customer'].dt.year
ds_new_var["Age"] = now.year - ds["Year_Birth"]

ds_new_var["TotalAcceptedCampaigns"] = ds["AcceptedCmp1"] + ds["AcceptedCmp2"] + ds["AcceptedCmp3"] + ds["AcceptedCmp4"] + ds["AcceptedCmp5"]
#Total amount spent
ds_new_var["TotalMoneySpent"] = ds["MntWines"] + ds["MntFruits"] + ds["MntMeatProducts"] + ds["MntFishProducts"] + ds["MntSweetProducts"] + ds["MntGoldProds"]
#Calculating the ratios of money spent per category
ds_new_var["RatioWines"] = ds["MntWines"] / ds_new_var["TotalMoneySpent"]
ds_new_var["RatioFruits"] = ds["MntFruits"] / ds_new_var["TotalMoneySpent"]
ds_new_var["RatioMeatProducts"] = ds["MntMeatProducts"] / ds_new_var["TotalMoneySpent"]
ds_new_var["RatioFishProducts"] = ds["MntFishProducts"] / ds_new_var["TotalMoneySpent"]
ds_new_var["RatioSweetProducts"] = ds["MntSweetProducts"] / ds_new_var["TotalMoneySpent"]
ds_new_var["RatioGoldProds"] = ds["MntGoldProds"] / ds_new_var["TotalMoneySpent"]

#Changing income to 2 years
ds_new_var["Income2Years"] = ds["Income"] * 2

#Calculating Effort Rate
ds_new_var["EffortRate"] = ds_new_var["TotalMoneySpent"]/ds_new_var["Income2Years"]

#All kids
ds_new_var["TotalKids"] = ds["Teenhome"] + ds["Kidhome"]

#People per Household
status = ["Together", "Married"]
ds_new_var["Count_Household"] = 0
ds_new_var["Count_Household"].loc[(ds["Marital_Status"].isin(status))] = 2 + ds_new_var["TotalKids"]
ds_new_var["Count_Household"].loc[(~ds["Marital_Status"].isin(status))] = 1 + ds_new_var["TotalKids"]

#Income per person in household
ds_new_var["Income_Per_Person"] = ds_new_var["Income2Years"]/ds_new_var["Count_Household"]
ds_new_var.head(5)

C:\Users\Filipa\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,RatioFruits,RatioMeatProducts,RatioFishProducts,RatioSweetProducts,RatioGoldProds,Income2Years,EffortRate,TotalKids,Count_Household,Income_Per_Person
ID,,,,,,,,,,,,,,,,,,,,,
6862,1971,Graduation,Divorced,1730.0,0,0,2014-05-18,65,1,1,...,0.125000,0.375000,0.125000,0.125000,0.125000,3460.0,0.002312,0,1,3460.000000
5376,1979,Graduation,Married,2447.0,1,0,2013-01-06,42,1,1,...,0.000578,0.997110,0.000578,0.000578,0.000578,4894.0,0.353494,1,3,1631.333333
11110,1973,Graduation,Single,3502.0,1,0,2013-04-13,56,2,1,...,0.200000,0.200000,0.000000,0.000000,0.200000,7004.0,0.000714,1,2,3502.000000
9931,1963,PhD,Married,4023.0,1,1,2014-06-23,29,5,0,...,0.000000,0.111111,0.111111,0.111111,0.111111,8046.0,0.001119,2,4,2011.500000
10311,1969,Graduation,Married,4428.0,0,1,2013-10-05,0,16,4,...,0.011142,0.033426,0.005571,0.011142,0.894150,8856.0,0.040537,1,3,2952.000000


#### Feature selection:

In [52]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)

In [97]:
clf.fit(#later)